In [1]:
import pandas as pd
from API_Key import key, username, password
import psycopg2 as pg2
import json
from datetime import datetime

In [2]:
conn = pg2.connect(database='TFT', user=username, password=password)
#c = conn.cursor()
df = pd.read_sql(sql=("select * from matches"),con=conn)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7928 entries, 0 to 7927
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   match            7928 non-null   object 
 1   game_datetime    7928 non-null   float64
 2   game_length      7928 non-null   float64
 3   game_version     7928 non-null   object 
 4   gold_left        7928 non-null   int64  
 5   last_round       7928 non-null   int64  
 6   level            7928 non-null   int64  
 7   placement        7928 non-null   int64  
 8   puuid            7928 non-null   object 
 9   time_eliminated  7928 non-null   float64
 10  total_damage     7928 non-null   int64  
 11  traits           7928 non-null   object 
 12  units            7928 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 805.3+ KB


In [4]:
df.head()

,match,game_datetime,game_length,game_version,gold_left,last_round,level,placement,puuid,time_eliminated,total_damage,traits,units
0,NA1_4107203113,1.637190e+12,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,0,34,8,3,2-xYFcIJUem__n9qL8bVWDc7uuuvvnp-SuB4VqKmGiPBFw...,1896.383179,77,"[{""name"": ""Set6_Bruiser"", ""num_units"": 2, ""sty...","[{""character_id"": ""TFT6_Vi"", ""items"": [], ""nam..."
1,NA1_4107203113,1.637190e+12,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,0,27,7,7,rIB0yQoA9jT06yokBJjP0wQQRbXe2xwZK1dlIrFwi471Im...,1550.396729,37,"[{""name"": ""Set6_Academy"", ""num_units"": 2, ""sty...","[{""character_id"": ""TFT6_Graves"", ""items"": [], ..."
2,NA1_4107203113,1.637190e+12,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,1,33,8,5,WiQACHT1c3gcMNkdAVHuRybWz48Dsbu3PbTZGlr28dOcCV...,1842.735352,91,"[{""name"": ""Set6_Academy"", ""num_units"": 1, ""sty...","[{""character_id"": ""TFT6_KogMaw"", ""items"": [12,..."
3,NA1_4107203113,1.637190e+12,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,1,30,8,6,FqFDLL5R9eVnnGE5TVK-kd1LiZ8GXeS7Ejy73BSnjddxz5...,1695.539795,41,"[{""name"": ""Set6_Academy"", ""num_units"": 1, ""sty...","[{""character_id"": ""TFT6_Swain"", ""items"": [2193..."
4,NA1_4107203113,1.637190e+12,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,0,27,8,8,4uM1ZdIf5gv4G-I9ZvP9zKajQsJurn24wQQP-UivRpLnh4...,1546.322754,53,"[{""name"": ""Set6_Academy"", ""num_units"": 1, ""sty...","[{""character_id"": ""TFT6_Darius"", ""items"": [], ..."


In [5]:
df['game_datetime'] = df['game_datetime'].apply(
                                            lambda x: datetime.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
df = df[df['game_datetime'] >= "2021-11-04"]

### Successfully webscraped champion names and saved them in pickle to prevent constant scraping

In [7]:
# # Getting champions from Latest Set
# from bs4 import BeautifulSoup
# import requests
# url = 'https://app.mobalytics.gg/tft/champions'
# soup = BeautifulSoup(requests.get(url).content, 'html.parser')

# champs = []
# for champ in soup.find("div", class_='m-1o47yso').find_all('a'):
#     champs.append(champ['href'])

# # removing excess string in champion name
# for position, champ in enumerate(champs):
#     champs[position] = champ.replace("/tft/champions/", "")

In [8]:
# import pickle 
# # Store data (serialize)
# with open('Set6_Champions.pickle', 'wb') as handle:
#     pickle.dump(champs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# Load data (deserialize)
import pickle
with open('Set6_Champions.pickle', 'rb') as handle:
    unserialized_data = pickle.load(handle)

champs = unserialized_data

In [11]:
# creating new columns for each champ in the TFT set and filling values with 0
for champ in champs:
    df[champ] = 0

In [14]:
df['traits'] = df['traits'].apply(lambda x: json.loads(x))
df['units'] = df['units'].apply(lambda x: json.loads(x))

In [15]:
active_traits = []
for match in df.traits:
    active_traits.append([{trait['name']:trait['tier_current']} 
                          for trait in match 
                          if trait['tier_current'] >=1])

In [17]:
df['active_traits'] = active_traits

In [18]:
items = list()
carry = list()
for row in df['units']:
    for resources in row:
        if len(resources['items']) >=1:
            items.append(resources['items'])
            carry.append(resources['name'])

In [19]:
army = []
for unit in df['units']:
    champs = []
    for champ in unit:
        champs.append(champ['character_id'][5:])
    army.append(champs) 

In [20]:
df['army'] = army

In [21]:
df.columns

Index(['match', 'game_datetime', 'game_length', 'game_version', 'gold_left',
       'last_round', 'level', 'placement', 'puuid', 'time_eliminated',
       'total_damage', 'traits', 'units', 'akali', 'blitz', 'braum', 'caitlyn',
       'camille', 'chogath', 'darius', 'dr-mundo', 'ekko', 'ezreal', 'fiora',
       'galio', 'gangplank', 'garen', 'graves', 'heimerdinger', 'illaoi',
       'janna', 'jayce', 'jhin', 'jinx', 'kaisa', 'kassadin', 'katarina',
       'kogmaw', 'leona', 'lissandra', 'lulu', 'lux', 'malzahar',
       'miss-fortune', 'orianna', 'poppy', 'quinn', 'samira', 'seraphine',
       'shaco', 'singed', 'sion', 'swain', 'tahm-kench', 'talon', 'taric',
       'tristana', 'trundle', 'twisted-fate', 'twitch', 'urgot', 'veigar',
       'vex', 'vi', 'viktor', 'warwick', 'yone', 'yuumi', 'zac', 'ziggs',
       'zilean', 'zyra', 'active_traits', 'army'],
      dtype='object')

In [22]:
# Need to rename some of the champion named columns in order to be more consistent
df.rename(columns={"dr-mundo": "drmundo", 
                   "blitz": "blitzcrank",
                  "tahm-kench": "tahmkench",
                  "twisted-fate": "twistedfate",
                  "miss-fortune": "missfortune"}, inplace=True)
df['rainingdummy']=0

In [44]:
#
for unit in df['army']:
    for champion in unit:
        df[champion.lower()].apply(lambda x: x+1)

KeyboardInterrupt: 

In [49]:
def champion_counter(df, row):
    for unit in df.iloc[row]['army']:
        df.iloc[row][unit.lower()] +=1
    return df

In [50]:
for num in range(len(df)):
    champion_counter(df, num)

<ipython-input-49-56d84c4411ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[row][unit.lower()] +=1


In [51]:
df.head()

,match,game_datetime,game_length,game_version,gold_left,last_round,level,placement,puuid,time_eliminated,...,warwick,yone,yuumi,zac,ziggs,zilean,zyra,active_traits,army,rainingdummy
0,NA1_4107203113,2021-11-17 23:03:32,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,0,34,8,3,2-xYFcIJUem__n9qL8bVWDc7uuuvvnp-SuB4VqKmGiPBFw...,1896.383179,...,0,0,0,0,0,0,0,"[{'Set6_Bruiser': 1}, {'Set6_Chemtech': 1}, {'...","[Vi, Lissandra, Taric, DrMundo, Orianna, Janna...",0
1,NA1_4107203113,2021-11-17 23:03:32,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,0,27,7,7,rIB0yQoA9jT06yokBJjP0wQQRbXe2xwZK1dlIrFwi471Im...,1550.396729,...,0,0,0,0,0,0,0,"[{'Set6_Academy': 1}, {'Set6_Enforcer': 1}, {'...","[Graves, Caitlyn, Kassadin, Garen, Blitzcrank,...",0
2,NA1_4107203113,2021-11-17 23:03:32,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,1,33,8,5,WiQACHT1c3gcMNkdAVHuRybWz48Dsbu3PbTZGlr28dOcCV...,1842.735352,...,0,0,0,0,0,0,0,"[{'Set6_Arcanist': 1}, {'Set6_Bruiser': 2}, {'...","[KogMaw, Lissandra, Malzahar, ChoGath, DrMundo...",0
3,NA1_4107203113,2021-11-17 23:03:32,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,1,30,8,6,FqFDLL5R9eVnnGE5TVK-kd1LiZ8GXeS7Ejy73BSnjddxz5...,1695.539795,...,0,0,0,0,0,0,0,"[{'Set6_Colossus': 1}, {'Set6_Cuddly': 1}, {'S...","[Swain, Talon, Seraphine, Sion, Yuumi, Galio]",0
4,NA1_4107203113,2021-11-17 23:03:32,2028.263306,Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...,0,27,8,8,4uM1ZdIf5gv4G-I9ZvP9zKajQsJurn24wQQP-UivRpLnh4...,1546.322754,...,0,0,0,0,0,0,0,"[{'Set6_Assassin': 2}, {'Set6_Bodyguard': 1}, ...","[Darius, TwistedFate, Shaco, Ekko, Braum, DrMu...",0


In [55]:
df.iloc[0]['vi'] = 1

<ipython-input-55-ec469ceb560a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0]['vi'] = 1


In [56]:
df.iloc[0]['vi']

0